Installing dependencies

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import torch.nn.functional as F
import gradio as gr
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
from tqdm import tqdm, trange

Loading fine-tuned model

In [ ]:
# Uncomment lines below and input path of model if you want to use a fine-tuned model

# tokenizer = AutoTokenizer.from_pretrained("gpt2")
# model = torch.load('path', map_location ='cpu')

Loading Pre-trained model

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = AutoModelForCausalLM.from_pretrained("gpt2")

Generation function

In [ ]:
#Inputs: prompt:string, sentence_length:int, num_sequences:int
#Outputs: generated_text:string
def generate_text(prompt, sentence_length, num_sequences):
    
    # Tokenize the prompt
    input_ids = tokenizer.encode(prompt, return_tensors='pt')
    
    # Generate text
    generated_text = model.generate(
        input_ids,
        max_length=sentence_length * num_sequences,
        min_new_tokens = 7,
        do_sample=True,
        top_p=0.95,
        top_k=50,
        pad_token_id=tokenizer.eos_token_id
    )
    
    # Decode the generated text
    generated_text = tokenizer.decode(generated_text[0], skip_special_tokens=True)
    
    return generated_text

#Inputs: generated_text:string
#Outputs: filtered_sentences:array of strings
def split_by_sentences(generated_text):
    # Split text by '.' character
    sentences = sent_tokenize(generated_text)
    # Remove incomplete sentences
    filtered_sentences = [sentence for sentence in sentences if sentence.endswith('.')]
    # Convert array of sentences into string and add newline
    filtered_sentences = '\n'.join([sentence for sentence in filtered_sentences])
    return filtered_sentences

Gradio Interface

In [ ]:
description = 'This is a text generation model which uses the transformer architecture to generate text given an prompt'
title = 'Story Generator'

text_gen_UI = gr.Interface(generate_text,
                  inputs=['text', gr.inputs.Slider(0,100, label='Number of generated words'), gr.inputs.Slider(0,20, label='Number of sequences')], 
                  outputs=[gr.outputs.Textbox()])
text_clean_UI = gr.Interface(split_by_sentences,
                  inputs=['text'], 
                  outputs=[gr.outputs.Textbox()])

gr.Series(text_gen_UI, text_clean_UI, description=description, title=title).launch()